In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.12,-134.97,78.67,66,24,7.99,few clouds
1,1,Puerto Ayora,EC,-0.74,-90.35,82.40,61,0,8.05,clear sky
2,2,Ketchikan,US,55.34,-131.65,42.80,93,90,19.46,moderate rain
3,3,Ushuaia,AR,-54.80,-68.30,55.40,69,40,21.92,light rain
4,4,Jumla,NP,29.27,82.18,37.27,55,8,3.69,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.12,-134.97,78.67,66,24,7.99,few clouds
1,1,Puerto Ayora,EC,-0.74,-90.35,82.40,61,0,8.05,clear sky
10,10,Toma,BF,12.77,-2.88,79.32,18,0,6.64,clear sky
12,12,Porto Velho,BR,-8.76,-63.90,77.00,94,40,2.24,thunderstorm
13,13,Hilo,US,19.73,-155.09,80.60,61,20,6.93,light rain
18,18,Faanui,PF,-16.48,-151.75,80.42,76,6,17.00,clear sky
19,19,Yulara,AU,-25.24,130.99,78.80,39,64,19.46,broken clouds
21,21,Banjar,ID,-8.19,114.97,78.80,94,75,3.36,broken clouds
24,24,Coahuayana,MX,18.73,-103.68,80.60,32,5,9.17,clear sky
25,25,Saint-Philippe,RE,-21.36,55.77,77.00,69,0,17.22,light rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                208
City                   208
Country                207
Lat                    208
Lng                    208
Max Temp               208
Humidity               208
Cloudiness             208
Wind Speed             208
Current Description    208
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
new_preferred_cities_df = preferred_cities_df.dropna()
new_preferred_cities_df.count()

City_ID                207
City                   207
Country                207
Lat                    207
Lng                    207
Max Temp               207
Humidity               207
Cloudiness             207
Wind Speed             207
Current Description    207
dtype: int64

In [70]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = new_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.67,few clouds,-23.12,-134.97,
1,Puerto Ayora,EC,82.40,clear sky,-0.74,-90.35,
10,Toma,BF,79.32,clear sky,12.77,-2.88,
12,Porto Velho,BR,77.00,thunderstorm,-8.76,-63.90,
13,Hilo,US,80.60,light rain,19.73,-155.09,
...,...,...,...,...,...,...,...
674,Manokwari,ID,81.07,moderate rain,-0.87,134.08,
677,Robertsport,LR,80.73,clear sky,6.75,-11.37,
679,Bom Jesus,BR,78.24,moderate rain,-9.07,-44.36,
683,Altamira,BR,75.81,overcast clouds,-3.20,-52.21,


In [85]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
idx = 0
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.iloc[idx, -1] = hotels["results"][0]["name"]
        print(hotels["results"][0])

    except:
        print("Hotel not found... skipping.")
    idx = idx + 1

{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -23.1206197, 'lng': -134.9687529}, 'viewport': {'northeast': {'lat': -23.1194598197085, 'lng': -134.9673886697085}, 'southwest': {'lat': -23.1221577802915, 'lng': -134.9700866302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': "Pension Maro'i", 'opening_hours': {'open_now': True}, 'photos': [{'height': 2304, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111098332724302207930">luigia b.</a>'], 'photo_reference': 'ATtYBwLdjoIPmVf09a0mA4Ixr2mdEF7jLSzLXCPttZEGmATT1grn5c1DxJEYWBKXlff26uJOxTkfEcjaEdcGEyad_6f7o2Ms9IVsZtS814cg7iNz-jWHlqbOzN5Wgtm-x43YA220w-EdLyRv5QnRMJx1F620rRZSvJ69547NJVxrRA2_6z_c', 'width': 4096}], 'place_id': 'ChIJnW6Jh0aIJ54RV2HnC1r0Hjw', 'plus_code': {'global_code': '5487V2HJ+QF'}, 'rating': 4, 'reference': 'ChIJnW6Jh0aIJ54RV2HnC1r0Hjw', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'user_ratings_to

{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 18.7010493, 'lng': -103.6609645}, 'viewport': {'northeast': {'lat': 18.7024599302915, 'lng': -103.6595256697085}, 'southwest': {'lat': 18.6997619697085, 'lng': -103.6622236302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hotel Los Arcos', 'photos': [{'height': 2448, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104051135651454298895">Marcos Perez</a>'], 'photo_reference': 'ATtYBwJ1bRYkCZGaQL9rdBHyy8BhM0F_XgRNzEyUW_mED9iP0MOGNQtQI_ToAphnA2KoQwM5xjAkTXnR5SYJNc57OkB1TJ86JFcNRODZybnY3WBB-iSTQlDam4L1lGC_OLIZHuLorwrX9tF1yzVPdKBmR2Laoli-_ckfzMhpp0iNidbyB50O', 'width': 3264}], 'place_id': 'ChIJrQ22ZmuiOoQRyNBKnp_tZgs', 'plus_code': {'compound_code': 'P82Q+CJ Coahuayana de Hidalgo, Michoacán, Mexico', 'global_code': '75CRP82Q+CJ'}, 'rating': 3.3, 'reference': 'ChIJrQ22ZmuiOoQRyNBKnp_tZgs', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest',

{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -0.7462424, 'lng': -48.0260077}, 'viewport': {'northeast': {'lat': -0.745239619708498, 'lng': -48.0246718697085}, 'southwest': {'lat': -0.747937580291502, 'lng': -48.0273698302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hotel do Luttis', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2448, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102344021618391815314">Alcione de Jesus</a>'], 'photo_reference': 'ATtYBwIBSnaLJBXUx6xLf-DRbk0QFwqPSOoWn9AWXQtj0Duz91lFrQCnMwM7hxvU0IypfO5-emlIi80-9etyMtJB3KFhNVEWP8rCdGtkxGpUdKtkr42LbLwotL8pVeX5_gw3qHrJZiAHP-s7Qil284w_BjRNGH8iQVlnSbvhQK7bf82FM8mP', 'width': 3264}], 'place_id': 'ChIJW9VNKrljppIRGTVWebXyyFU', 'plus_code': {'compound_code': '7X3F+GH São Caetano de Odivelas - State of Pará, Brazil', 'global_code': '68FH7X3F+GH'}, 'rating': 3.8, 'reference': 'ChIJW9VNKrljppIRGTVWebXyyFU', 'scope': 'G

{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -0.1920404, 'lng': -74.7820258}, 'viewport': {'northeast': {'lat': -0.190675969708498, 'lng': -74.78062966970849}, 'southwest': {'lat': -0.193373930291502, 'lng': -74.78332763029151}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'la casona de juancho', 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/114654050989237696743">Mateo Diaz</a>'], 'photo_reference': 'ATtYBwL-kvZ3INJ-vS3UkLGu11iioTjS88Cnl4KBa4nizdoMjSWlxyb99oIUpwpBvE_T6uDLxqnO9Z1Ns7vfFpPJWJ0JttgDWELwDPtI-1VahuEiwhUo-VK6bA9kWSBlPfeRtt6i6z6bI84A1lUYBfOVgYvXhb7fxYQigrVfrVEeAuxb99kc', 'width': 5376}], 'place_id': 'ChIJodPxXWJP35ERBTGtIcVF9iw', 'plus_code': {'global_code': '67F7R659+55'}, 'rating': 4.6, 'reference': 'ChIJodPxXWJP35ERBTGtIcVF9iw', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'user_ratings_total': 93, 'vicinity': 'Unnam

{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -14.2536624, 'lng': 50.15776460000001}, 'viewport': {'northeast': {'lat': -14.2522504197085, 'lng': 50.1591427802915}, 'southwest': {'lat': -14.2549483802915, 'lng': 50.1564448197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Melrose', 'photos': [{'height': 4512, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/103897448784342326757">Thierry Bernard</a>'], 'photo_reference': 'ATtYBwIaSJE8XhwhOzQHZHA0UM3am_-CXuBX3ToP7igAlH-KOGe_NOPuhcvHoyzRrO12HkFH0pI4zkmYp0sUxwDAVQUnOX4qdc5Kpn1yFLlezNXxS0kTKca3PIkL_Mnaoowcfu3jv2K2m2F7ApUxcMFmrPrQlimvepU-40XzBD4F7w1JGdGJ', 'width': 6016}], 'place_id': 'ChIJ3foVdL7xPSIREzr3ukv3zts', 'plus_code': {'compound_code': 'P5W5+G4 Sambava, Madagascar', 'global_code': '5HQGP5W5+G4'}, 'rating': 3.1, 'reference': 'ChIJ3foVdL7xPSIREzr3ukv3zts', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'us

...

{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -12.0567302, 'lng': 134.2316741}, 'viewport': {'northeast': {'lat': -12.0552456697085, 'lng': 134.2332699302915}, 'southwest': {'lat': -12.0579436302915, 'lng': 134.2305719697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Dhukurrdji Lodge', 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115321001241227174245">Alexandra Retzer</a>'], 'photo_reference': 'ATtYBwLHyawMgxuOPfXMFBXZrMmjNyeHIexszYhEGbuh6BNkVrQU6yy2yD2iQqHKvfahAUTnLotsZcq1IKRBhZVyn4oaO2y906AnjSkEImuTPd5dsL1FcCwulhAwNwDrEo_aMCOeicYy7gRII9DLpjFfFShX6MZ1uM1O4p7-yfx93o0dZWRJ', 'width': 4032}], 'place_id': 'ChIJlXpNmJOItSwR2G1QujWTTZk', 'plus_code': {'compound_code': 'W6VJ+8M Maningrida NT, Australia', 'global_code': '5QVPW6VJ+8M'}, 'rating': 4.3, 'reference': 'ChIJlXpNmJOItSwR2G1QujWTTZk', 'scope': 'GOOGLE', 'types': ['meal_takeaway', 'lodging', 'restaurant', 

{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -0.9221483, 'lng': 122.8642974}, 'viewport': {'northeast': {'lat': -0.920786269708498, 'lng': 122.8656434802915}, 'southwest': {'lat': -0.9234842302915021, 'lng': 122.8629455197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Penginapan Kim', 'place_id': 'ChIJpwp7Dqilhi0REXnMdp0nICs', 'plus_code': {'compound_code': '3VH7+4P Biak, Banggai Regency, Central Sulawesi, Indonesia', 'global_code': '6QF43VH7+4P'}, 'reference': 'ChIJpwp7Dqilhi0REXnMdp0nICs', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'vicinity': 'Jalan Trans Sulawesi, Biak'}
{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -23.5262125, 'lng': 148.1602479}, 'viewport': {'northeast': {'lat': -23.52495696970849, 'lng': 148.1615461802915}, 'southwest': {'lat': -23.5276549302915, 'lng': 148.1588482197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_ap

{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 1.619946, 'lng': 18.0647877}, 'viewport': {'northeast': {'lat': 1.621486230291502, 'lng': 18.06608743029151}, 'southwest': {'lat': 1.618788269708498, 'lng': 18.0633894697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Préfecture de la Likouala', 'photos': [{'height': 548, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105431641604092110691">charmock awabeney</a>'], 'photo_reference': 'ATtYBwJJfY4a_x6UjHSK6aiKYDssIo0_wxX149wexx9LZj_BwTX5cX31nMqojFsNnNxvaweowch2tsjsQiY8g29aXUU2bREWIo50SY31w3wT-R4x5IQLxqsGmD-1bUg1jq9i5OvsvAtvonrxS1Vc7BhAH_-TA9yqxx2vDWdK8bgV9pe5w7es', 'width': 712}], 'place_id': 'ChIJSxK-GPB5pBARBzey3pN0Gg0', 'plus_code': {'compound_code': 'J397+XW Impfondo, Republic of the Congo', 'global_code': '6FHWJ397+XW'}, 'reference': 'ChIJSxK-GPB5pBARBzey3pN0Gg0', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'

{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -12.7207158, 'lng': -38.3308402}, 'viewport': {'northeast': {'lat': -12.7193132197085, 'lng': -38.3294909197085}, 'southwest': {'lat': -12.7220111802915, 'lng': -38.3321888802915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Sítio Blasi', 'place_id': 'ChIJH1BDvPFpFgcRQ47ekNaP6U8', 'plus_code': {'compound_code': '7MH9+PM Camaçari - State of Bahia, Brazil', 'global_code': '59V37MH9+PM'}, 'reference': 'ChIJH1BDvPFpFgcRQ47ekNaP6U8', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'vicinity': 'Rua Rio Madeira - loteamento las palmas 7, Camaçari'}
Hotel not found... skipping.
{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': -22.659695, 'lng': 167.443914}, 'viewport': {'northeast': {'lat': -22.65830361970849, 'lng': 167.4450214802915}, 'southwest': {'lat': -22.6610015802915, 'lng': 167.4423235197085}}}, 'icon': 'https://maps

{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 0.3873627, 'lng': 9.754987999999997}, 'viewport': {'northeast': {'lat': 0.3893221802915021, 'lng': 9.756314080291501}, 'southwest': {'lat': 0.386624219708498, 'lng': 9.753616119708497}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Hôtel Santa Barbara', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1080, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109221747438974452078">ASSOUA joel</a>'], 'photo_reference': 'ATtYBwKPsfSqMUgan1OgQHrQ5X9yWItgF1WSfcGbquYtvgcDVeo6VcgSClGnJuI2vwmz8_mkrzFmJ_0VtpAIpJPiWkZTgnVhyjcD3tSTIjlDM0jIxC7fLG24WOJ-sHOz9bUEyz1jG-x6cGG8sbLy9wgN2RZ53TSUsvc-uMNiZO7f9hYObxe8', 'width': 1920}], 'place_id': 'ChIJKySmPGgOfxARAWvjU-5ZpyI', 'plus_code': {'compound_code': '9QP3+WX Ntoum, Gabon', 'global_code': '6FGF9QP3+WX'}, 'rating': 3.2, 'reference': 'ChIJKySmPGgOfxARAWvjU-5ZpyI', 'scope': 'GOOGLE', 'types': ['lodging', 'po

In [86]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.67,few clouds,-23.12,-134.97,Pension Maro'i
1,Puerto Ayora,EC,82.40,clear sky,-0.74,-90.35,Finch Bay Galapagos Hotel
10,Toma,BF,79.32,clear sky,12.77,-2.88,Auberge de l'amitié
12,Porto Velho,BR,77.00,thunderstorm,-8.76,-63.90,Hotel Vila Rica Porto Velho
13,Hilo,US,80.60,light rain,19.73,-155.09,Hilo Hawaiian Hotel
...,...,...,...,...,...,...,...
674,Manokwari,ID,81.07,moderate rain,-0.87,134.08,Swiss Belhotel Manokwari
677,Robertsport,LR,80.73,clear sky,6.75,-11.37,Kwepunha Retreat
679,Bom Jesus,BR,78.24,moderate rain,-9.07,-44.36,Hotel Brasão & Floricultura Bella Flor
683,Altamira,BR,75.81,overcast clouds,-3.20,-52.21,Requinte Hotel


In [91]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.67,few clouds,-23.12,-134.97,Pension Maro'i
1,Puerto Ayora,EC,82.40,clear sky,-0.74,-90.35,Finch Bay Galapagos Hotel
10,Toma,BF,79.32,clear sky,12.77,-2.88,Auberge de l'amitié
12,Porto Velho,BR,77.00,thunderstorm,-8.76,-63.90,Hotel Vila Rica Porto Velho
13,Hilo,US,80.60,light rain,19.73,-155.09,Hilo Hawaiian Hotel
18,Faanui,PF,80.42,clear sky,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
19,Yulara,AU,78.80,broken clouds,-25.24,130.99,Desert Gardens Hotel - Ayers Rock Resort
21,Banjar,ID,78.80,broken clouds,-8.19,114.97,The Hamsa Resort
24,Coahuayana,MX,80.60,clear sky,18.73,-103.68,Hotel Los Arcos
25,Saint-Philippe,RE,77.00,light rain,-21.36,55.77,"Chambres d'hôte ""La Trinité"""


In [92]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [93]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [94]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))